In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install nearpy
from nearpy import Engine
from nearpy.hashes import RandomBinaryProjections
import numpy as np
import os
import cv2
import dlib
from sklearn.model_selection import train_test_split


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 661.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 279.9/279.9 kB 3.0 MB/s eta 0:00:00


In [4]:
def extract_face_landmarks(image):
    # Yüz tespiti işlemleri
    detector = dlib.get_frontal_face_detector()
    predictor = dlib.shape_predictor(shape_predictor)

    gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    faces = detector(gray)

    if len(faces) > 0:
        landmarks = predictor(gray, faces[0])
        return np.array([[p.x, p.y] for p in landmarks.parts()])
    else:
        return None


In [5]:
# Veri setinin bulunduğu klasörü belirtin
data_dir = '/content/drive/MyDrive/Colab Notebooks/flw'

In [6]:
# Görüntüleri ve etiketleri depolamak için boş listeler oluşturun
images = []
labels = []

# Tüm klasörleri döngüye alarak görüntüleri yükleyin ve etiketleri oluşturun
for person_name in os.listdir(data_dir):
    person_dir = os.path.join(data_dir, person_name)
    if os.path.isdir(person_dir):
        for image_name in os.listdir(person_dir):
            image_path = os.path.join(person_dir, image_name)
            image = cv2.imread(image_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            images.append(image)
            labels.append(person_name)

class_names = np.unique(labels)
class_indices = {class_name: i for i, class_name in enumerate(class_names)}

# Etiketleri sayısal forma dönüştürme
labels = [class_indices[label] for label in labels]

X_train, X_val, y_train, y_val = train_test_split(images, labels, test_size=0.2, random_state=42)

KeyboardInterrupt: ignored

In [ ]:
# Resize the images after defining X_train
X_train_resized = [cv2.resize(image, (250, 250)) for image in X_train]
X_val_resized = [cv2.resize(image, (250, 250)) for image in X_val]

# Convert to numpy arrays
X_train = np.array(X_train_resized)
y_train = np.array(y_train)
X_val = np.array(X_val_resized)
y_val = np.array(y_val)

# Yüz tespiti için dlib kullanma
detector = dlib.get_frontal_face_detector()

# Specify the path to the shape_predictor_68_face_landmarks.dat file
shape_predictor = "/content/drive/MyDrive/Colab Notebooks/shape_predictor_68_face_landmarks.dat"


In [ ]:
# Yüz özelliklerini çıkartma
X_train_landmarks = [extract_face_landmarks(image) for image in X_train]
X_val_landmarks = [extract_face_landmarks(image) for image in X_val]


In [ ]:
# Boş olan özellikleri çıkartılan listeden temizleme
X_train = [X_train[i] for i in range(len(X_train_landmarks)) if X_train_landmarks[i] is not None]
y_train = [y_train[i] for i in range(len(X_train_landmarks)) if X_train_landmarks[i] is not None]
X_val = [X_val[i] for i in range(len(X_val_landmarks)) if X_val_landmarks[i] is not None]
y_val = [y_val[i] for i in range(len(X_val_landmarks)) if X_val_landmarks[i] is not None]
X_train_landmarks = [X_train_landmarks[i] for i in range(len(X_train_landmarks)) if X_train_landmarks[i] is not None]
X_val_landmarks = [X_val_landmarks[i] for i in range(len(X_val_landmarks)) if X_val_landmarks[i] is not None]


In [ ]:
lat_features_from_training_set = []

# Önceki eğitim setinizdeki tüm özellik vektörlerini düzleştirin ve flat_features_from_training_set listesine ekleyin
for landmarks in X_train_landmarks:
    flat_features = landmarks.reshape(-1)
    flat_features_from_training_set.append(flat_features)

# Listeyi numpy array'e çevirin
flat_features_from_training_set = np.array(flat_features_from_training_set)


In [ ]:
# Önceki eğitim setinden çıkarılan özellik vektörleri
flat_features_from_training_set = []

# Önceki eğitim setinizdeki tüm özellik vektörlerini düzleştirin ve flat_features_from_training_set listesine ekleyin
for landmarks in X_train_landmarks:
    flat_features = landmarks.reshape(-1)
    flat_features_from_training_set.append(flat_features)

# Listeyi numpy array'e çevirin
flat_features_from_training_set = np.array(flat_features_from_training_set)

# LSH için gerekli işlemleri yapma
dimension = flat_features_from_training_set.shape[1]  # Özellik vektör boyutu
num_hashes = 10  # LSH hash sayısı


In [ ]:
# LSH motorunu oluşturma
lshashes = [RandomBinaryProjections('rbp', num_projections=num_hashes) for _ in range(num_hashes)]
engine = Engine(dimension, lshashes=lshashes)

# Önceki eğitim setinden çıkarılan özellik vektörlerini LSH motoruna ekleme
for i, flat_features in enumerate(flat_features_from_training_set):
    engine.store_vector(flat_features, i)


In [ ]:
# Yeni bir görüntü için yüz özelliklerini çıkartma
new_image_path = '/content/drive/MyDrive/Colab Notebooks/WIN_20231208_02_55_04_Pro.jpg'
new_image = cv2.imread(new_image_path)
new_image = cv2.cvtColor(new_image, cv2.COLOR_BGR2RGB)
new_image_resized = cv2.resize(new_image, (250, 250))
new_landmarks = extract_face_landmarks(new_image_resized)


In [ ]:
# Boş olmayan bir özellik vektörü varsa ve yüz tespiti başarılı olduysa
if new_landmarks is not None:
    # Yüz özelliklerini düzleştirme
    new_flat_features = new_landmarks.reshape(-1)

    # LSH motorunu kullanarak benzer yüzleri bulma
    your_distance_threshold = 0.8
    results = engine.neighbours([new_flat_features])

    # İlk 5 benzer yüzü yazdırma
    print("En yakın 5 yüz:")
    count = 0
    for result in results:
        index = result[1]  # Benzer yüzün indisini al
        print(f"Uzaklık: {result[2]}")
        count += 1
        if count >= 5:
            break
else:
    print("Yüz tespiti başarısız.")